---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [55]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [56]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [57]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df = pd.read_csv('university_towns.txt', sep="/n", header=None)
    df.columns = ['data']
    df.data = df.data.str.replace(r'\[.*', '')
    df.data = df.data.str.replace(r'\(.*', '')
     
    new_df = pd.DataFrame(columns = ["State", "RegionName"])
    
    for index, row in df.iterrows():
        #print(row['data'])
        if row['data'] in states.values():
            currentState = row['data'].strip()
        else:
            new_df = new_df.append({"State" : currentState , "RegionName" : row['data'].strip()}, ignore_index=True)

    return new_df

#get_list_of_university_towns()

In [58]:
# From Bureau of Economic Analysis, US Department of Commerce, 
# the GDP over time of the United States in current dollars (use the chained value in 2009 dollars), 
# in quarterly intervals, in the file gdplev.xls. For this assignment, only look at GDP data from the 
# first quarter of 2000 onward.

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', skiprows=8, header=None, usecols=[4,6])
    df.columns = ['quarter', 'GDP']
    index_of_2000 = df[df.quarter == '2000q1'].index[0]
    df = df[df.index >= index_of_2000]
    
    # A recession is defined as 
    # starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
    
    # Quarters where previous quarter had higher GDP and next quarter has yet lower GDP 
    # take the most recent one and return only the "quarter" column string value
    df = df[(df.GDP < df.GDP.shift(1)) & (df.GDP > df.GDP.shift(-1))].head(1).quarter.values[0]
    
    return df

#get_recession_start()

In [59]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', skiprows=8, usecols=[4,6])
    df.columns = ['quarter', 'GDP']
    year_2000_index = df[df.quarter == '2000q1'].index[0]
    
    df = df[df.index >= year_2000_index]
    
    rec_start_date = get_recession_start()
    rec_start_date_index = df[df.quarter == rec_start_date].index[0]
    #print(rec_start_date_index)
    # "ending with two consecutive quarters of GDP growth"
    # Mistake: I misunderstood the definition of the "recession end".
    # It ends upon the _second_ consecutive quarter of GDP growth (not the first quarter, as I originally thought)
    
    # STILL NEED TO FIX....it should be 2009q4
    
    # This returns q2
    #df = df[(df.GDP.shift(-1) > df.GDP) & (df.GDP.shift(-1) < df.GDP.shift(-2))]
    
    # This returns q3
    #df = df[(df.GDP.shift(1) < df.GDP) & (df.GDP < df.GDP.shift(-1)) & (df.GDP.shift(-1) < df.GDP.shift(-2))]
    
    # Previous period needs to be less than current period & two-periods-ago needs to be less than previous-period
    df = df[(df.GDP.shift(1) < df.GDP) & (df.GDP.shift(2) < df.GDP.shift(1))]
    
    # AND must be after a recession began
    df = df[df.index > rec_start_date_index]
    
    # Get the earliest quarter
    df = df.head(1).quarter.values[0]
    
    
    
    
    return df

#get_recession_end()

In [60]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls', skiprows=8, usecols=[4, 6])
    df.columns = ['quarter', 'GDP']
    year_2000_index = df[df.quarter == '2000q1'].index[0]
    df = df[df.index >= year_2000_index]
    
    start_string = get_recession_start()
    start_index = df[df.quarter == start_string].index[0]
    
    end_string = get_recession_end()
    end_index = df[df.quarter == end_string].index[0]
    
    recession_df = df[(df.index <= end_index) & (df.index >= start_index)]
    
    return recession_df.sort_values('GDP').head(1).quarter.values[0]
    
    #return recession_df

get_recession_bottom()

'2009q2'

In [61]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    years = [1996, 1997, 1998, 1999]
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    del_list = []
    for year in years:
        for month in months:
            newString = str(year) + '-' + month
            if newString in ['1996-01', '1996-02', '1996-03']:
                continue
            del_list.append(newString)
            
    df.drop(del_list, axis=1, inplace=True)
    df.sort_values('State', inplace=True)
    df = df.set_index(['State', 'RegionName'])
    df.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1, inplace=True)
    
    df = df.groupby(np.arange(len(df.columns))//3, axis=1).mean()
    
    years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
    quarters = ['q1', 'q2', 'q3', 'q4']
    newCols = []
    for year in years:
        for quarter in quarters:
            newString = str(year) + quarter
            if newString == '2016q4':
                continue
            newCols.append(newString)
            
    df.columns = newCols
    
    df = df.rename(index=states)
    
    return df

#convert_housing_data_to_quarters().loc['Texas'].loc['Austin'].loc['2010q3']
#convert_housing_data_to_quarters()

In [62]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    
    df = convert_housing_data_to_quarters()
    
    # We're only interested in the two columns that represent the start and bottom of the recession
    df = df[[rec_start, rec_bottom]]
    df.columns = ['start', 'bottom']
    
    df['change'] = df.bottom - df.start
    
    df = df.drop(['start', 'bottom'], axis=1)
    
    #df = df.rename(index=states)
    
    
    univ_towns = get_list_of_university_towns()
    univ_towns['isUnivTown'] = 1
    #univ_towns = univ_towns.set_index(['State', 'RegionName'])
    
    df = df.reset_index()
    
    # Mistakes made so far:
    # 1. There was leading/trailing whitespace in 'df' indices, needed to strip()
    # 2. I copy-pasted some code and forgot to update it on pasting
    
    
    
    univ_towns['State'] = univ_towns.State.str.strip()
    univ_towns['RegionName'] = univ_towns.RegionName.str.strip()
    
    df['State'] =df.State.str.strip()
    df['RegionName'] = df.RegionName.str.strip()
    
#     print(univ_towns.head())
#     print()
#     print(df.head())
    
    new_df = pd.merge(df, univ_towns, left_on = ['State', 'RegionName'], right_on=['State', 'RegionName'], how='left')
    
    #df.isUnivTown.fillna(value = 0, inplace=True)
    
    #new_df = new_df[new_df.isUnivTown == 1]
    
    new_df['isUnivTown'] = new_df['isUnivTown'].fillna(0)
    
    non_univ_change = new_df.where(new_df.isUnivTown == 0)['change']
    univ_change = new_df.where(new_df.isUnivTown == 1)['change']
    
    t_stat, p_value = ttest_ind(univ_change, non_univ_change, nan_policy='omit')
    
    mean_non_univ = non_univ_change.mean()
    mean_univ = univ_change.mean()
    
    if mean_univ > mean_non_univ:
        better = "university town"
    else: 
        better = "non-university town"
        
    if p_value < 0.01:
        different = True
    else:
        different = False
    
    
    return (different, p_value, better)

#run_ttest()